In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
training_exp_name = "training/10_23_23_Dense_WMH"

dfc = rs.load_configs(
    root / training_exp_name,
    properties=False,
)

In [ ]:
dfc

In [ ]:
%%yaml inference_config

log:
    root: '?'
    log_interval: 50
    track_image_level: True
    track_pixel_level: True 

model:
    exp_root : '?'
    num_workers: 1  

dataset:
    split: '?' 
    slicing: dense_full 
    input_type: volume 

calibration:
    binarize: True
    conf_interval_start: 0.5
    conf_interval_end: 1.0
    num_bins: 10
    neighborhood_width: 3
    square_diff: True

In [ ]:
%%yaml cal_metrics_cfg 

cal_metrics:
    - ECE:
        func: ese.experiment.metrics.ECE
    - TL_ECE:
        func: ese.experiment.metrics.TL_ECE
    - CW_ECE:
        func: ese.experiment.metrics.CW_ECE
    - LoMS:
        func: ese.experiment.metrics.LoMS
    - TL_LoMS:
        func: ese.experiment.metrics.TL_LoMS
    - CW_LoMS:
        func: ese.experiment.metrics.CW_LoMS

In [ ]:
from ionpy.util import dict_product, Config

# Need to define the experiment name
inference_exp_root = str(root / "inference/12_06_23_WMH_Debug")

# Get the inference options.
##################################################
dataset_options = {
    'log.root': [inference_exp_root],
    'model.exp_root': [str(root / training_exp_name)],
    'dataset.split': ['cal'],
}

base_cfg = Config(inference_config).update(cal_metrics_cfg)

cfgs = []
for cfg_update in dict_product(dataset_options):
    new_cfg = base_cfg.update(cfg_update)
    cfgs.append(new_cfg)

In [ ]:
from ese.experiment.analysis.inference import get_cal_stats
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

get_cal_stats(cfgs[0])

In [ ]:
# from ionpy.slite.submit import submit_jobs
# from ese.experiment.analysis.inference import get_cal_stats

# submit_jobs(
#     exp_root=inference_exp_root,
#     config_list=cfgs, 
#     job_func=get_cal_stats,
#     available_gpus=["0"]
#     )